# 뉴스 크롤링

In [8]:
pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.3/448.3 kB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.7 MB/s eta 0:00:00


In [9]:
pip install webdriver-manager

In [10]:
!apt-get update
!apt install chromium-chromedriver

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [46.6 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [49.8 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,279 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,455 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,419 kB]
Hit:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:13 https:/

In [11]:
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re
import datetime
from pytz import timezone
KST = timezone('Asia/Seoul')

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome(options=options)
# browser.implicitly_wait(1)

## 기사 링크 크롤링

In [88]:
# 기사 링크 크롤링
# 카테고리당 100개 = 5페이지

six_url_list = []   # url 저장 리스트
entertainment_url_list = []
sports_url_list = []

category_list = []
category_names = ["정치", "경제", "사회", "생활/문화", "세계", "IT/과학", "연예", "스포츠"]

def getSixLink():    # 정치, 경제, 사회, 생활/문화, 세계, IT/과학

    for category in range(3, 4):     # 6
        a_list = []
        for page in range(1, 2):  # 1, 6
            url = f'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1={100 + category}#&date=%2000:00:00&page={page}'
            browser.get(url)

            time.sleep(0.5)

            soup = BeautifulSoup(browser.page_source, "html.parser")
            a_list.extend(soup.select(".type06_headline dt+dt a"))
            a_list.extend(soup.select(".type06 dt+dt a"))

            print(f"{category_names[category]} {page} 페이지")

        for a in a_list:
            six_url_list.append(a["href"])
            category_list.append(category_names[category])

def getEntertainmentLink():   # 연예
    # today = str(datetime.datetime.now(KST))[:11]  # 서울 기준 시간
    a_list = []
    today = datetime.date.today()

    for page in range(1, 2):  # 1, 5
        url = f'https://entertain.naver.com/now#sid=106&date={today}&page={page}'
        browser.get(url)

        time.sleep(0.5)

        soup = BeautifulSoup(browser.page_source, "html.parser")

        a_list.extend(soup.select(".news_lst li>a"))


        print(f"연예 {page} 페이지")

    for a in a_list:
        entertainment_url_list.append("https://entertain.naver.com" + a["href"])
        category_list.append(category_names[6])

def getSportsLink():    # 스포츠  (페이지마다 개수가 달라서 6페이지를 이동)
    # today = str(datetime.datetime.now(KST))[:11].replace('-', '')  # 서울 기준 시간
    a_list = []
    today = str(datetime.date.today()).replace('-', '')

    for page in range(1, 2):  # 1, 7
        url = f'https://sports.news.naver.com/general/news/index?isphoto=N&type=latest&date={today}&page={page}'
        browser.get(url)

        time.sleep(0.5)

        soup = BeautifulSoup(browser.page_source, "html.parser")
        a_list.extend(soup.select(".news_list li>a"))

        print(f"스포츠 {page} 페이지")

    for i in range(len(a_list)):
        if i == 100:  # 100개 링크 추가했으면 멈추기
            break
        sports_url_list.append("https://sports.news.naver.com/news" + re.search('\?.+', a_list[i]["href"]).group())
        category_list.append(category_names[7])

# getSixLink()
# getEntertainmentLink()
# getSportsLink()


In [44]:
print("6개 카테고리 수집 개수 :", len(six_url_list))
print("6개 카테고리 수집 개수 :", len(set(six_url_list)))
print()
print("연예 카테고리 수집 개수 :", len(entertainment_url_list))
print("연예 카테고리 수집 개수 :", len(set(entertainment_url_list)))
print()
print("스포츠 카테고리 수집 개수 :", len(sports_url_list))
print("스포츠 카테고리 수집 개수 :", len(set(sports_url_list)))
print()
print("전체 카테고리 수집 개수 : ", len(category_list))

6개 카테고리 수집 개수 : 0
6개 카테고리 수집 개수 : 0

연예 카테고리 수집 개수 : 0
연예 카테고리 수집 개수 : 0

스포츠 카테고리 수집 개수 : 19
스포츠 카테고리 수집 개수 : 19

전체 카테고리 수집 개수 :  19


## 제목, 본문, 날짜 크롤링

In [45]:
# 수집한 태그 속에서 내용만 뽑아서 담는다.

def clean(article):   # 전처리
    article = re.sub('\w+ 기자','',article)
    article = re.sub('\w{2,4} 온라인 기자','',article)
    article = re.sub('\w{2,4}기자','',article)
    article = re.sub('사진','',article)
    article = re.sub('포토','',article)
    # article = re.sub('\S+@[a-z.]+','',article)

    article = re.sub('\n','',article)
    article = re.sub('\t','',article)
    article = re.sub('\u200b','',article)
    article = re.sub('\xa0','',article)
    article = re.sub('([a-zA-Z])','',article)
    article = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',article)
    article = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘’“”|\(\)\[\]\<\>`\'…》]','',article)
    article = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',article)

    return article


In [46]:
# 수집한 링크의 제목, 본문, 날짜 크롤링

title_list = []     # 제목
content_list = []   # 본문
date_list = []      # 날짜

def getSixContent(url_list):   # 정치, 경제, 사회, 생활/문화, 세계, IT/과학
    title = []
    content = []
    date = []

    for url in url_list:
        browser.get(url)

        time.sleep(0.5)

        soup = BeautifulSoup(browser.page_source, "html.parser")

        title.extend(soup.select("#title_area span"))                 # 제목

        c = soup.find_all(attrs={"id" : "dic_area"})                  # 본문

        while c[0].find(attrs={"class" : "end_photo_org"}):           # 이미지 있는 만큼
          c[0].find(attrs={"class" : "end_photo_org"}).decompose()    # 본문 이미지에 있는 글자 없애기

        while c[0].find(attrs={"class" : "vod_player_wrap"}):         # 영상 있는 만큼
          c[0].find(attrs={"class" : "vod_player_wrap"}).decompose()  # 본문 영상에 있는 글자 없애기

        content.extend(c)

        date.extend(soup.select("._ARTICLE_DATE_TIME"))               # 날짜

    for t in title:
        title_list.append(clean(t.text))

    for c in content:
        content_list.append(clean(c.text))

    for d in date:
        date_list.append(d.text)


def getEntertainmentContent(url_list):    # 연예
    title = []
    content = []
    date = []

    for url in url_list:
        browser.get(url)

        time.sleep(0.5)

        soup = BeautifulSoup(browser.page_source, "html.parser")

        title.extend(soup.select(".end_tit"))                 # 제목

        c = soup.find_all(attrs={"id" : "articeBody"})                # 본문

        while c[0].find(attrs={"class" : "end_photo_org"}):           # 이미지 있는 만큼
          c[0].find(attrs={"class" : "end_photo_org"}).decompose()    # 본문 이미지에 있는 글자 없애기
          if c[0].find(attrs={"class" : "caption"}):                  # 이미지 설명 없애기
            c[0].find(attrs={"class" : "caption"}).decompose()

        while c[0].find(attrs={"class" : "video_area"}):         # 영상 있는 만큼
          c[0].find(attrs={"class" : "video_area"}).decompose()  # 본문 영상에 있는 글자 없애기

        while c[0].find(attrs={"name" : "iframe"}):
           c[0].find(attrs={"name" : "iframe"}).decompose()

        content.extend(c)

        date.extend(soup.select_one(".author em"))               # 날짜

    for t in title:
      title_list.append(clean(t.text))

    for c in content:
      content_list.append(clean(c.text))

    for d in date:
      date_list.append(d.text)


def getSportsContent(url_list):   # 스포츠
    title = []
    content = []
    date = []

    for url in url_list:

        browser.get(url)            # 스포츠 기사만 여기서 넘어가는데 굉장히 진짜 대박 오래 걸림 왜 그럴까? 스포츠
        print(url)
        time.sleep(0.5)

        soup = BeautifulSoup(browser.page_source, "html.parser")

        title.extend(soup.select(".news_headline .title"))                 # 제목

        c = soup.find_all(attrs={"class" : "news_end"})                # 본문

        while c[0].find(attrs={"class" : "end_photo_org"}):           # 이미지 있는 만큼
          c[0].find(attrs={"class" : "end_photo_org"}).decompose()    # 본문 이미지에 있는 글자 없애기

        while c[0].find(attrs={"class" : "image"}):
           c[0].find(attrs={"class" : "image"}).decompose()

        while c[0].find(attrs={"class" : "vod_area"}):         # 영상 있는 만큼
          c[0].find(attrs={"class" : "vod_area"}).decompose()  # 본문 영상에 있는 글자 없애기

        c[0].find(attrs={"class" : "source"}).decompose()
        c[0].find(attrs={"class" : "byline"}).decompose()
        c[0].find(attrs={"class" : "reporter_area"}).decompose()
        c[0].find(attrs={"class" : "copyright"}).decompose()
        c[0].find(attrs={"class" : "categorize"}).decompose()
        c[0].find(attrs={"class" : "promotion"}).decompose()

        content.extend(c)

        date.extend(soup.select_one(".info span"))               # 날짜

    for t in title:
      title_list.append(clean(t.text))

    for c in content:
      content_list.append(clean(c.text))

    for d in date:
      date_list.append(d.text)

# getSixContent(six_url_list)
# getEntertainmentContent(entertainment_url_list)
getSportsContent(sports_url_list)


KeyboardInterrupt: ignored

In [ ]:
print(len(title_list))
print(len(content_list))
print(len(date_list))

1399
1399
1399


## 데이터 프레임 생성

In [ ]:
# 데이터 프레임으로 변환
all_url_list = six_url_list + entertainment_url_list + sports_url_list

article_df = pd.DataFrame({"category" : category_list,
                           "date" : date_list,
                           "title" : title_list,
                           "content" : content_list,
                           "url" : all_url_list})

article_df

NameError: ignored

In [ ]:
# article_df.to_csv("C:\\Users\\82106\\OneDrive\\바탕 화면\\article.csv", encoding='utf-8-sig')

# 전처리

**KoNLPy**
- 한국어 정보처리를 위한 파이썬 패키지
- https://konlpy.org/ko/latest/api/konlpy.tag/#okt-class


In [7]:
pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 47.7 MB/s eta 0:00:00


In [14]:
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import matplotlib.pyplot as plt


# CountVectorizer : 텍스트 빈도만을 카운트해서 벡터화
# TfidfVectorizer : 자주 나오는 단어에 높은 가중치, 모든 문서에서 자주 나오는 단어에 패널티

In [15]:
def getNouns():
    okt = Okt()
    nouns_list = []   # 명사 리스트

    for content in article_df["content"]:
        nouns_list.append(okt.nouns(content))     # 명사 추출, 리스트 반환

    article_df["nouns"] = nouns_list   # 데이터 프레임에 추가


getNouns()

NameError: ignored

In [16]:
# 명사까지 추출한 파일 불러오기, 1400개 기사

test_df = pd.read_csv("https://raw.githubusercontent.com/Yoon-juhan/naverNewsCrawling/main/article.csv")
test_df['nouns'] = test_df['nouns'].apply(lambda x: eval(x))        # 명사 열을 다시 리스트 형식으로 변환
test_df.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1, inplace=True)  # 파일 불러왔을 때 필요없이 생기는 열 삭제

In [17]:
# 영어 기사 삭제

empty_nouns = []
for i in range(len(test_df['nouns'])):
    if len(test_df['nouns'][i]) == 0:
        empty_nouns.append(i)

test_df.drop(empty_nouns, inplace=True)

In [18]:
test_df

,category,date,title,content,url,nouns
0,정치,2023.11.05. 오후 3:12,홍준표 곧 정계 빅뱅 올 것···나라도 살아야겠다는 엑소더스 친윤부터 퍼질지도,홍준표 대구시장은 곧 정계 빅뱅이 올 것으로 보인다라며 잘 대처하기를 바란다고 경고...,https://n.news.naver.com/mnews/article/657/000...,"[홍준표, 대구시, 곧, 정계, 빅뱅, 것, 대처, 경고, 홍, 시장, 자신, 나,..."
1,정치,2023.11.05. 오후 3:11,北 성공한 11월18일 미사일공업절’로그날 맞춰 도발 감행’ 우려도,한·미·일 군사협력에 “핵전쟁 발발·3차대전 이어진다” 겁박북한이 11월 18일을 ...,https://n.news.naver.com/mnews/article/005/000...,"[일, 군사, 협력, 핵전쟁, 발발, 차, 대전, 겁박, 북한, 미사일, 공업, 절..."
2,정치,2023.11.05. 오후 3:09,동앵과 뉴스터디“영남 스타 수도권으로” 발칵 뒤집힌 국민의힘,안녕하세요 동앵과 뉴스터디 동정민 앵커입니다국민의힘 혁신위원회가 출범하고 시끌시끌합...,https://n.news.naver.com/mnews/article/449/000...,"[동앵, 뉴, 스터디, 정민, 앵커, 국민, 힘, 혁신, 위원회, 출범, 제, 뉴스..."
3,정치,2023.11.05. 오후 3:09,민주당 이탄희 조선일보가 두려워하는 건 ○○정치,판사 출신 이탄희 의원 선거법 퇴행엔 “직을 걸고 막겠다”양당 구도를 깨자며 연합 ...,https://n.news.naver.com/mnews/article/006/000...,"[판사, 출신, 이탄, 의원, 선거법, 퇴행, 직, 당, 구도, 연합, 정치, 주장..."
4,정치,2023.11.05. 오후 2:59,정청래 尹 직격 대통령 잘못 뽑아 온 국민이 고생이대로 가면 나라 큰일 날 듯,정청래 더불어민주당 최고위원이 윤석열 정부를 비판하며 이재명 대표 중심으로 뭉친 민...,https://n.news.naver.com/mnews/article/002/000...,"[정청래, 민주당, 최고, 위원, 윤석열, 정부, 비판, 이재명, 대표, 중심, 민..."
...,...,...,...,...,...,...
1394,연예,2023.11.05. 오후 1:41,판타지 보이즈 신곡 안무 시안 영상으로 글로벌 영향력 확실히 입증,그룹 판타지 보이즈김규래·홍성민·오현태·이한빈·링치·강민서·히카리·소울·...,https://entertain.naver.com/now/read?oid=437&a...,"[그룹, 판타지, 보이즈, 김규, 홍성민, 현태, 이한빈, 링치, 강민서, 히카리,..."
1395,연예,2023.11.05. 오후 1:41,위대한 가이드’ 골동품 투어로 인생역전 성공할까,위대한 가이드’ 멤버들이 골동품 시장에서 숨은 보석 찾기에 나선다6일 방송되는 에브...,https://entertain.naver.com/now/read?oid=144&a...,"[가이드, 멤버, 골동품, 시장, 보석, 찾기, 방송, 에브리, 가이드, 고규필, ..."
1396,연예,2023.11.05. 오후 1:40,무인도의 디바 박은빈 차학연에 꽃다발 전달받고 눈물 펑펑,무인도의 디바’ 박은빈에게 의문의 꽃다발이 도착한다토일드라마무인도의 디바’극본 박...,https://entertain.naver.com/now/read?oid=109&a...,"[무인도, 디바, 박은빈, 의문, 꽃다발, 도착, 토, 일드, 라마, 무인도, 디바..."
1397,연예,2023.11.05. 오후 1:40,충주시장 5만원 넘는 선물 덥석 받았다가 진땀충주맨은 바로 거절,서울뉴스1 조길형 충주시장이 예능 프로그램에 출연해 17만원 상당의 옷을 선물받...,https://entertain.naver.com/now/read?oid=421&a...,"[서울, 뉴스, 조길형, 충주시, 예능, 프로그램, 출연, 상당, 옷, 선물, 진땀..."


In [19]:
# category_names = ["정치", "경제", "사회", "생활/문화", "세계", "IT/과학", "연예", "스포츠"]

vector_list = []

def getVector():    # 카테고리 별로 벡터 생성
    for i in range(7):
        text = [" ".join(noun) for noun in test_df[test_df['category'] == category_names[i]]['nouns']]  # 명사 열을 하나의 리스트에 담는다.

        tfidf_vectorizer = TfidfVectorizer(min_df = 3, ngram_range=(1, 5))
        tfidf_vectorizer.fit(text)
        vector = tfidf_vectorizer.transform(text).toarray()  # vector list 반환
        vector = np.array(vector)
        vector_list.append(vector)

getVector()

In [20]:
len(vector_list)

7

# 군집화

**DBSCAN**
- https://bcho.tistory.com/1205


In [21]:
from sklearn.cluster import DBSCAN

# DBSCAN : 밀도 기반 클러스터링
# 점 p에서 부터 거리 e (epsilon)내에 점이 m(minPts) 개 있으면 하나의 군집으로 인식

In [22]:
# 카테고리 별로 군집화
# cluster_number 열에 군집 번호 생성

cluster_number_list = []

for vector in vector_list:
    model = DBSCAN(eps=0.1, min_samples=1, metric='cosine')
    result = model.fit_predict(vector)
    cluster_number_list.extend(result)

test_df['cluster_number'] = cluster_number_list

In [ ]:
# 카테고리 별로 군집의 개수를 센다.

cluster_counts_df = pd.DataFrame({'category' : [""],
                                  'cluster_number' : [0],
                                  'cluster_count' : [0]})

for i in range(7):
    t = test_df[test_df['category'] == category_names[i]]['cluster_number'].value_counts().reset_index()
    t.columns = ['cluster_number', 'cluster_count']
    t['category'] = [category_names[i]] * len(t)

    cluster_counts_df = pd.concat([cluster_counts_df, t])

cluster_counts_df = cluster_counts_df[cluster_counts_df['cluster_count'] != 0]
cluster_counts_df

In [79]:
# 상위 군집 10개씩만 추출

cluster_counts_df = cluster_counts_df[cluster_counts_df.index < 10]
cluster_counts_df

,category,cluster_number,cluster_count
0,정치,100,3
1,정치,9,3
2,정치,93,2
3,정치,113,2
4,정치,56,2
...,...,...,...
5,연예,80,5
6,연예,47,5
7,연예,28,5
8,연예,81,5


In [ ]:
for i in range(len(cluster_counts_df)):
    category_name, cluster_number = cluster_counts_df.iloc[i, 0:2]    # 카테고리 이름, 군집 번호
    print(test_df[(test_df['category'] == category_name) & (test_df['cluster_number'] == cluster_number)]['title'])